In [18]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn.linear_model  as sk  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score

import pandas as pd
from helper import *
import tensorflow as tf

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

# Stuff that needs to be done

- Find another framework.
    - Sklearn
    - Spark
    - Tensorflow/R
- Algorithms
    - Linear Regression
    - Logistic Regression
    - Maybe SVM /Random forest
- Find two metrics 
    - RMSE 
    - Some other
- Do 70/30 split training and 10 cross validation
    - only did cross validation
- Find another dataset 
    - Sum data with noise 
    -  Maybe housing data set
    

## Import DataSet

In [19]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [20]:
get_data(all_urls) # retrieves the data if there is no data folder

In [21]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/housing dataset.csv" # has more than 30 features
# need one more
# what a brilliant idea to name files with space

In [22]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

## Load datasets sum_noise

In [23]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise.head(n=1)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number


## Load datasets housing

In [24]:
housing_price = pd.read_csv(housing_price_path)
housing_price.head(n=1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [25]:
# Remove 'Instance' as it simply represents the row number
sumdata_noise.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise[['Noisy Target']]

# Extract 'Nosiy Target Class' as regression target
sumdata_noise_classif_Y = sumdata_noise['Noisy Target Class']

# Extract rest columns as explananatory variables
sumdata_noise_X = sumdata_noise.iloc[:, 1:-2]


### Linear Regression using pyspark

In [26]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


Exception: Java gateway process exited before sending the driver its port number

In [ ]:
def spark_linear_regression_cross(data_df):
    
    
    input_features = list(sumdata_noise.columns)[:-2]
    ouput_label = list(sumdata_noise.columns)[-2]
    
    data = data_df.select(input_features)\
      .rdd\
      .map(lambda line:LabeledPoint(line[-1],line[1:-1]))\
      .toDF()
    
  
    as_ml = udf(lambda v: v.asML() if v is not None else None, VectorUDT())
    dataset = data.withColumn("features", as_ml("features"))
    
    lr = LinearRegression()
    grid = ParamGridBuilder().addGrid(lr.maxIter, [0,1]).build()
    evaluator = RegressionEvaluator(metricName="rmse")
    
    cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator, numFolds = 10)
    
    cvModel = cv.fit(dataset)
    print("Root Mean Squared Error = " + str(cvModel.avgMetrics[0]))
    
  
    

##### http://www.techpoweredmath.com/spark-dataframes-mllib-tutorial/

In [ ]:
def spark_linear_regression(data_df):
    """
        This doesnt have cross validation 
    """
    input_features = list(sumdata_noise.columns)[:-2]
    ouput_label = list(sumdata_noise.columns)[-2]
    
    data = data_df.select(input_features)\
      .rdd\
      .map(lambda line:LabeledPoint(line[-1],line[1:-1]))\
      .toDF()
    
    # converting RDD to a dataframe
    as_ml = udf(lambda v: v.asML() if v is not None else None, VectorUDT())
    result = data.withColumn("features", as_ml("features"))
    result
    
    lr = LinearRegression()

    # Fit 2 models, using different regularization parameters
    modelA = lr.fit(result, {lr.regParam:0.0})
    modelB = lr.fit(result, {lr.regParam:100.0})
    
    
    predictionsA = modelA.transform(result)
    
    evaluator = RegressionEvaluator(metricName="rmse")
    RMSE = evaluator.evaluate(predictionsA)
    print("ModelA: Root Mean Squared Error = " + str(RMSE))
    
    predictionsB = modelB.transform(result)
    RMSE = evaluator.evaluate(predictionsB)
    print("ModelB: Root Mean Squared Error = " + str(RMSE))
    
    

In [ ]:
for chunk in data_chunks:

    if chunk > sumdata_noise.shape[0]: # if chunk is greater than the no. of examples
        break
        
    data = sumdata_noise[:chunk]
    data.drop('Noisy Target Class', axis=1)
    data_df = sqlContext.createDataFrame(data)
     
    
    spark_linear_regression_cross(data_df)
    print("\n")


### Linear regression using scikit learn

In [27]:
def sklearn_linear_regression_cross( data_x, data_y,  data_set, algorithm=sk.LinearRegression): 
    
    algo_name = algorithm.__name__
    
    for chunk in data_chunks:

        if chunk > sumdata_noise.shape[0]: # if chunk is greater than the no. of examples
            break

        X = data_x.head(n = chunk)
        y = data_y.head(n = chunk)
        
        X = X.as_matrix()
        y = y.as_matrix() 
        kf = KFold(n_splits=10)
        errors = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            lm = sk.LinearRegression()
            model = lm.fit(X_train, y_train)
            predictions = lm.predict(X_test) 
        
            errors.append(mean_squared_error(y_test, predictions)**0.5)  # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
        #http://statweb.stanford.edu/~tibs/sta306bfiles/cvwrong.pdf
        
        error = sum(errors)/len(errors)
        print ("Data set: {} Algorithm: {}  Chunk Size: {} Error: {}".format(data_set,
                                                                          algo_name,
                                                                          chunk,
                                                                           error
                                                                         ))
 
              

In [28]:

sklearn_linear_regression_cross(sumdata_noise_X, sumdata_noise_reg_Y,"Noisy Sum", sk.LinearRegression )

Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 100 Error: 97493.4018149379
Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 500 Error: 86679.40047927969
Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 1000 Error: 82856.35758829128
Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 5000 Error: 83374.4764714345
Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 10000 Error: 83734.8448468906
Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 50000 Error: 83493.96774110087
Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 100000 Error: 83808.93090170946
Data set: Noisy Sum Algorithm: LinearRegression  Chunk Size: 500000 Error: 83617.59069544692


### Logistic Regression using Keras

In [58]:
#1D log reg
def keras1DLogReg(X_train, Y_train, X_test, Y_test, input_dimensions):
    #Reshape and normalise inputs
    X_train = X_train.reshape(len(X_train), input_dimensions)
    X_test = X_test.reshape(len(X_test), input_dimensions)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255

    #Convert class vectors to binary class matrices
    nb_classes=0
    from keras.utils import np_utils 
    Y_train = np_utils.to_categorical(Y_train, nb_classes)
    Y_test = np_utils.to_categorical(Y_test, nb_classes)

    #Build the model
    output_dim = 10
    model = Sequential()
    model.add(Dense(output_dim, input_dim = input_dim, activation='softmax'))
    batch_size = 128
    epochs = 20

    #Compile ze model
    model.compile(optimizer = 'sgd', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    history = model.fit(X_train, 
                        Y_train, 
                        batch_size = batch_size,
                        epochs = epochs,
                        verbose = 1,
                        validation_data=(X_test, Y_test))
    score = model.evaluate(X_test, Y_test, verbose = 0)

    print ('Test score: ', score[0])
    print ('Test accuracy', score[1])

In [ ]:
#Logistic Regression using MNIST dataset
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
keras1DLogReg(X_train, y_train, X_test, Y_test, 784)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s - loss: 1.2938 - acc: 0.6811 - val_loss: 0.8151 - val_acc: 0.8291
Epoch 2/20
60000/60000 [==============================] - 1s - loss: 0.7201 - acc: 0.8383 - val_loss: 0.6089 - val_acc: 0.8613
Epoch 3/20
60000/60000 [==============================] - 1s - loss: 0.5898 - acc: 0.8580 - val_loss: 0.5265 - val_acc: 0.8737
Epoch 4/20
22784/60000 [==========>...................] - ETA: 0s - loss: 0.5456 - acc: 0.8659